In [1]:
import cv2
import numpy as np

def Gaussian_Window(x,y) :                                                                
    d = np.sqrt(x*x+y*y)                                                                   
    sigma, mu = 15.0, 0.0
    gaussian_window = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) ) / (np.sqrt(2*np.pi)*sigma)
    gaussian_window = gaussian_window / np.sum(gaussian_window)
    
    return gaussian_window

def Harris_Corner_Detection(img) :                                ## Corner 검출 함수
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img_sobel_x = cv2.Sobel(img_gray, cv2.CV_32F, 1, 0)            ## 이미지를 1차 미분하기 위한 x방향의 sobel_filter
    img_sobel_y = cv2.Sobel(img_gray, cv2.CV_32F, 0, 1)            ## 이미지를 1차 미분하기 위한 y방향의 sobel_filter

    IxIx = img_sobel_x * img_sobel_x                               ## Harris Corner detection matrix의 value값 생성
    IyIy = img_sobel_y * img_sobel_y
    IxIy = img_sobel_x * img_sobel_y

    a,b = np.meshgrid(np.linspace(-1,1,5), np.linspace(-1,1,5))
    gaussian_window = Gaussian_Window(a,b)                          ## Gaussian window를 생성(위에 method call)

    offset = int(gaussian_window.shape[0]/2)                        ## window size만큼 계산하기 위한 범위 설정

    R = np.zeros(img_gray.shape)                                    ## 1차원 상의 eigen value값인 R을 초기화

    for y in range(offset, height-offset):
        for x in range(offset, width-offset):
            ## 5x5 window와 미분한 이미지 matrix를 곱함
            window_IxIx = np.dot(gaussian_window, IxIx[y-offset:y+offset+1, x-offset:x+offset+1])
            window_IyIy = np.dot(gaussian_window, IyIy[y-offset:y+offset+1, x-offset:x+offset+1])
            window_IxIy = np.dot(gaussian_window, IxIy[y-offset:y+offset+1, x-offset:x+offset+1])
            
            Mxx = window_IxIx.sum()                                 ## matrix 내 모든값 더함 (Sigma 수행)
            Myy = window_IyIy.sum()
            Mxy = window_IxIy.sum()

            det = Mxx*Myy - Mxy*Mxy                                 ## R을 구하기 위해서 determinant와 trace 선언
            trace = Mxx + Myy
            
            R[y,x] = det - 0.04 * (trace ** 2)                      ## 모든 pixel에 R값을 만듦
    
    
    cv2.normalize(R, R, 0.0, 1.0, cv2.NORM_MINMAX)                   ## 모든 R 값을 0과 1사이로 nomalization
    th = 0.3
    for y in range(offset, height-offset):                          
        for x in range(offset, width-offset):
            if R[y, x] > th :                                        ## threshold가 0.3 이상인 값들만 Corner로 검출
                img.itemset((y, x, 0), 0)                             
                img.itemset((y, x, 1), 0)
                img.itemset((y, x, 2), 255)                          ## Corner인 곳만 빨간색으로 출력되게 설정
    
    return img

if __name__ == '__main__' :
    img = cv2.imread('opencv/picture/palace.jpg', cv2.IMREAD_COLOR)
    img = cv2.resize(img, (640,480))
    
    global height, width
    height, width = img.shape[:2]
    
    result = Harris_Corner_Detection(img)
    
    cv2.imshow("Harris Corner Detector", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    ## R값을 잘 파악해서 threshold를 설정해야하는 문제가 있다.

In [61]:
import cv2
import numpy as np

img = cv2.imread('opencv/picture/palace.jpg')
img = cv2.resize(img, (640,480))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray = np.float32(img_gray)

## cv2.cornerHarris(입력 이미지, 코너검출을 위한 이웃pixel 범위, sobel filter 인자값, R constant)
dst = cv2.cornerHarris(img_gray, 5, 3, 0.24)

dst = cv2.dilate(dst,None)


img[dst>0.15*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
